## Comparison to yfinance

In [1]:
import os

import pandas as pd
from IPython.core.interactiveshell import InteractiveShell

from yahooquery import Ticker
import yfinance as yf

InteractiveShell.ast_node_interactivity = "all"
pd.options.display.float_format = '{:,}'.format
pd.options.display.max_columns = None
pd.set_option('display.max_rows', 100)

In [2]:
symbol = 'aapl'

YF_aapl = yf.Ticker('aapl')
YQ_aapl = Ticker('aapl')

### Financials

#### yfinance

In [3]:
%%time
attrs = ['cashflow', 'balance_sheet', 'financials']
yf_d = {}
for attr in attrs:
    df = getattr(YF_aapl, attr)
    yf_d[attr] = df.shape[0]

Wall time: 3.15 s


#### yahooquery

In [4]:
%%time
methods = ['cash_flow', 'balance_sheet', 'income_statement']
yq_d = {}
for method in methods:
    df = getattr(YQ_aapl, method)()
    df.set_index(['asOfDate', 'periodType'], inplace=True)
    df = df.T
    yq_d[method] = df.shape[0]

Wall time: 1.34 s


In [5]:
# Comparison of the number of items returned for each
cash_flow = yf_d.pop('cashflow')
income_statement = yf_d.pop('financials')
yf_d.update({'cash_flow': cash_flow, 'income_statement': income_statement})
pd.DataFrame({'yahooquery': yq_d, 'yfinance': yf_d})

,yahooquery,yfinance
cash_flow,26,19
balance_sheet,66,25
income_statement,39,22


Yahooquery retrieves more cash flow, balance sheet, and income statement data than yfinance and quicker!

### Historical Pricing

In [6]:
# Compare downloads for all companies within the S&P500
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
sp500 = tables[0]['Symbol'].tolist()
sp500 = [symbol.replace(".", "-") for symbol in sp500]

#### yfinance

In [7]:
%%time
yf_data = yf.download(sp500, period='ytd', interval='1d', group_by='ticker')

[*********************100%***********************]  505 of 505 completed
Wall time: 56.1 s


In [8]:
type(yf_data)
yf_data.shape
yf_data.head()

pandas.core.frame.DataFrame

(181, 3030)

FDX                                        \
                         Open               High                Low   
Date                                                                  
2020-01-02  152.4199981689453 155.10000610351562 151.60000610351562   
2020-01-03              152.5 153.33999633789062 151.83999633789062   
2020-01-06             151.25  153.5500030517578  150.1699981689453   
2020-01-07 153.36000061035156 155.22000122070312 152.33999633789062   
2020-01-08 154.57000732421875             157.75 153.92999267578125   

                                                                          SO  \
                        Close          Adj Close   Volume               Open   
Date                                                                           
2020-01-02 155.10000610351562  153.1490936279297  3027600 63.650001525878906   
2020-01-03 153.17999267578125 151.25320434570312  2216500 62.529998779296875   
2020-01-06  153.3000030517578 151.37171936035156  2389700 62.560001373291016   
2020-01-07  154.8000030517578  152.8528289794922  2140100 62.560001373291016   
2020-01-08  157.1300048828125 155.15353393554688  3053200  62.70000076293945   

                                                                    \
                        High                Low              Close   
Date                                                                 
2020-01-02 63.66999816894531  62.41999816894531 62.619998931884766   
2020-01-03 62.90999984741211  62.43000030517578 62.560001373291016   
2020-01-06              63.0   62.5099983215332 62.810001373291016   
2020-01-07 62.72999954223633   62.2400016784668 62.619998931884766   
2020-01-08 62.84000015258789 62.380001068115234 62.599998474121094   

                                                      UNP                     \
                    Adj Close   Volume               Open               High   
Date                                                                           
2020-01-02  60.62262725830078  4675900  180.9499969482422  182.3800048828125   
2020-01-03 60.564544677734375  3531800  179.5399932861328 181.19000244140625   
2020-01-06  60.80656814575195  3507600  179.6999969482422  180.0800018310547   
2020-01-07  60.62262725830078  3191700 179.22999572753906  179.5800018310547   
2020-01-08  60.60326385498047  3413600 177.88999938964844  181.0500030517578   

                                                                              \
                          Low              Close          Adj Close   Volume   
Date                                                                           
2020-01-02 179.97000122070312 182.27000427246094  179.2933349609375  2444800   
2020-01-03  178.2899932861328 180.97999572753906 178.02438354492188  2344200   
2020-01-06 177.94000244140625 179.42999267578125 176.49969482421875  3142500   
2020-01-07 177.61000061035156 178.07000732421875  175.1619110107422  4096400   
2020-01-08 177.47999572753906 180.02000427246094      177.080078125  2817500   

                          CCL                                        \
                         Open               High                Low   
Date                                                                  
2020-01-02 51.349998474121094  51.66999816894531  50.72999954223633   
2020-01-03  50.04999923706055 50.400001525878906  49.56999969482422   
2020-01-06 49.349998474121094 49.400001525878906  48.27000045776367   
2020-01-07  48.13999938964844  48.66999816894531 48.130001068115234   
2020-01-08               48.5 49.029998779296875  48.43000030517578   

                                                                         LW  \
                        Close          Adj Close   Volume              Open   
Date                                                                          
2020-01-02 51.310001373291016 50.718055725097656  4201000  85.7699966430664   
2020-01-03  49.88999938964844  49.31443405151367  4892300  87.1500015258789   
2020-01-0

#### yahooquery

In [9]:
%%time
tickers = Ticker(sp500, asynchronous=True)
yq_data = tickers.history(period='ytd', interval='1d')

Wall time: 22.1 s


In [10]:
type(yq_data)
yq_data.shape
yq_data.head()

pandas.core.frame.DataFrame

(91280, 8)

open                low      volume  \
symbol date                                                           
MMM    2020-01-02 177.67999267578125 177.13999938964844 3,601,700.0   
       2020-01-03 177.02000427246094  175.6300048828125 2,466,900.0   
       2020-01-06 177.14999389648438 176.35000610351562 1,998,000.0   
       2020-01-07 178.27999877929688 176.82000732421875 2,173,000.0   
       2020-01-08              178.0 177.64999389648438 2,758,300.0   

                                high              close           adjclose  \
symbol date                                                                  
MMM    2020-01-02 180.00999450683594              180.0  175.0394744873047   
       2020-01-03 178.66000366210938  178.4499969482422  173.5321807861328   
       2020-01-06  178.7100067138672  178.6199951171875 173.69749450683594   
       2020-01-07 178.50999450683594 177.89999389648438 172.99732971191406   
       2020-01-08              181.5  180.6300048828125 175.65211486816406   

                   dividends  splits  
symbol date                           
MMM    2020-01-02        0.0     0.0  
       2020-01-03        0.0     0.0  
       2020-01-06        0.0     0.0  
       2020-01-07        0.0     0.0  
       2020-01-08        0.0     0.0

### Option Chain

#### yfinance

In [11]:
%%time
dataframes = []
for option in YF_aapl.options:
    options = YF_aapl.option_chain(option)
    calls = options[0]
    calls['optionType'] = 'calls'
    puts = options[1]
    puts['optionType'] = 'puts'
    dataframes.append(pd.concat([calls, puts]))
yf_option_chain = pd.concat(dataframes)

Wall time: 4.53 s


In [12]:
yf_option_chain

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,optionType
0,AAPL200925C00057500,2020-09-18 16:45:29,57.5,49.65,0.0,0.0,0.0,0.0,22.0,0.0,1.0000000000000003e-05,True,REGULAR,USD,calls
1,AAPL200925C00058750,2020-09-21 04:11:42,58.75,50.75,nan,nan,0.0,0.0,nan,nan,0.0,True,REGULAR,USD,calls
2,AAPL200925C00060000,2020-09-16 16:24:35,60.0,47.25,0.0,0.0,0.0,0.0,1.0,0.0,1.0000000000000003e-05,True,REGULAR,USD,calls
3,AAPL200925C00061250,2020-09-11 17:22:11,61.25,49.3,0.0,0.0,0.0,0.0,2.0,0.0,1.0000000000000003e-05,True,REGULAR,USD,calls
4,AAPL200925C00062500,2020-09-18 16:40:11,62.5,44.6,0.0,0.0,0.0,0.0,5.0,0.0,1.0000000000000003e-05,True,REGULAR,USD,calls
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,AAPL230120P00140000,2020-09-21 04:01:39,140.0,42.0,0.0,0.0,0.0,0.0,nan,1.0,1.0000000000000003e-05,True,REGULAR,USD,puts
19,AAPL230120P00150000,2020-09-21 04:01:39,150.0,55.1,0.0,0.0,0.0,0.0,nan,41.0,1.0000000000000003e-05,True,REGULAR,USD,puts
20,AAPL230120P00155000,2020-09-21 04:01:39,155.0,57.6,0.0,0.0,0.0,0.0,nan,62.0,1.0000000000000003e-05,True,REGULAR,USD,puts
21,AAPL230120P00160000,2020-09-21 04:01:39,160.0,59.2,0.0,0.0,0.0,0.0,nan,2.0,1.0000000000000003e-05,True,REGULAR,USD,puts


#### yahooquery

In [13]:
%%time
yq_option_chain = YQ_aapl.option_chain

Wall time: 1.57 s


In [14]:
yq_option_chain

contractSymbol  strike currency  lastPrice  \
symbol expiration optionType                                                    
aapl   2020-09-25 calls       AAPL200925C00057500    57.5      USD      49.65   
                  calls       AAPL200925C00058750   58.75      USD      50.75   
                  calls       AAPL200925C00060000    60.0      USD      47.25   
                  calls       AAPL200925C00061250   61.25      USD       49.3   
                  calls       AAPL200925C00062500    62.5      USD       44.6   
...                                           ...     ...      ...        ...   
       2023-01-20 puts        AAPL230120P00140000   140.0      USD       42.0   
                  puts        AAPL230120P00150000   150.0      USD       55.1   
                  puts        AAPL230120P00155000   155.0      USD       57.6   
                  puts        AAPL230120P00160000   160.0      USD       59.2   
                  puts        AAPL230120P00200000   200.0      USD       97.8   

                              change  percentChange  volume  openInterest  \
symbol expiration optionType                                                
aapl   2020-09-25 calls          0.0            0.0    22.0           0.0   
                  calls          0.0            0.0     0.0           0.0   
                  calls          0.0            0.0     1.0           0.0   
                  calls          0.0            0.0     2.0           0.0   
                  calls          0.0            0.0     5.0           0.0   
...                              ...            ...     ...           ...   
       2023-01-20 puts           0.0            0.0     0.0           1.0   
                  puts           0.0            0.0     0.0          41.0   
                  puts           0.0            0.0     0.0          62.0   
                  puts           0.0            0.0     0.0           2.0   
                  puts           0.0            0.0     0.0           6.0   

                              bid  ask contractSize       lastTradeDate  \
symbol expiration optionType                                              
aapl   2020-09-25 calls       0.0  0.0      REGULAR 2020-09-18 16:45:29   
                  calls       0.0  0.0      REGULAR 2020-09-21 04:11:42   
                  calls       0.0  0.0      REGULAR 2020-09-16 16:24:35   
                  calls       0.0  0.0      REGULAR 2020-09-11 17:22:11   
                  calls       0.0  0.0      REGULAR 2020-09-18 16:40:11   
...                           ...  ...          ...                 ...   
       2023-01-20 puts        0.0  0.0      REGULAR 2020-09-21 04:01:39   
                  puts        0.0  0.0      REGULAR 2020-09-21 04:01:39   
                  puts        0.0  0.0      REGULAR 2020-09-21 04:01:39   
                  puts        0.0  0.0      REGULAR 2020-09-21 04:01:39   
                  puts        0.0  0.0      REGULAR 2020-09-21 04:01:39   

                                  impliedVolatility  inTheMoney  
symbol expiration optionType                                     
aapl   2020-09-25 calls      1.0000000000000003e-05        True  
                  calls                         0.0        True  
                  calls      1.0000000000000003e-05        True  
                  calls      1.0000000000000003e-05        True  
                  calls      1.0000000000000003e-05        True  
...                                             ...         ...  
       2023-01-20 puts       1.0000000000000003e-05        True  
                  puts       1.0000000000000003e-05        True  
                  puts       1.0000000000000003e-05        True  
                  puts       1.0000000000000003e-05        True  
                  puts       1.0000000000000003e-05        True  

[4135 rows x 14 columns]

Retrieving all option expiration dates is a simple one-liner in yahooquery (and faster)!

### Multiple Symbols

In [15]:
symbols = 'FB AAPL AMZN NFLX GOOG'

#### yfinance

In [16]:
%%time
attrs = [
    'info', 'financials', 'quarterly_financials', 'major_holders',
    'institutional_holders', 'balance_sheet', 'quarterly_balance_sheet',
    'cashflow', 'quarterly_cashflow', 'earnings', 'quarterly_earnings',
    'sustainability', 'recommendations', 'calendar'
]
yf_d = {}
tickers = yf.Tickers(symbols)
for symbol in symbols.split(" "):
    ticker = getattr(tickers.tickers, symbol)
    yf_d[symbol] = {}
    for attr in attrs:
        yf_d[symbol][attr] = getattr(ticker, attr)

Wall time: 13.6 s


In [17]:
yf_d.keys()
yf_d['FB'].keys()

dict_keys(['FB', 'AAPL', 'AMZN', 'NFLX', 'GOOG'])

dict_keys(['info', 'financials', 'quarterly_financials', 'major_holders', 'institutional_holders', 'balance_sheet', 'quarterly_balance_sheet', 'cashflow', 'quarterly_cashflow', 'earnings', 'quarterly_earnings', 'sustainability', 'recommendations', 'calendar'])

#### yahooquery

In [18]:
%%time
tickers = Ticker(symbols, asynchronous=True)
data = tickers.all_modules

Wall time: 371 ms


In [20]:
data['FB'].keys()

dict_keys(['assetProfile', 'recommendationTrend', 'cashflowStatementHistory', 'indexTrend', 'defaultKeyStatistics', 'industryTrend', 'quoteType', 'incomeStatementHistory', 'fundOwnership', 'summaryDetail', 'insiderHolders', 'calendarEvents', 'upgradeDowngradeHistory', 'price', 'balanceSheetHistory', 'earningsTrend', 'secFilings', 'institutionOwnership', 'majorHoldersBreakdown', 'balanceSheetHistoryQuarterly', 'earningsHistory', 'esgScores', 'summaryProfile', 'netSharePurchaseActivity', 'insiderTransactions', 'sectorTrend', 'incomeStatementHistoryQuarterly', 'cashflowStatementHistoryQuarterly', 'earnings', 'pageViews', 'financialData'])

Again, retrieving data, this time with multiple symbols, is faster and simpler!